In [14]:

class Statement:
    def __init__(self, subject, predicate, singular=True):
        self.subject = subject
        self.predicate = predicate
        self.singular = singular

class Relation:
    def __init__(self):
        self.map = {}

    def add(self, key, subject, predicate, singular=True):
        self.map[key] = Statement(subject, predicate, singular)

    def print_statement(self, key):
        st = self.map[key]
        if st.singular:
            print(f"{st.subject} is {st.predicate}", end="")
        else:
            print(f"{st.subject} are {st.predicate}", end="")

    def print_neg_statement(self, key):
        st = self.map[key]
        if st.singular:
            print(f"{st.subject} is not {st.predicate}", end="")
        else:
            print(f"{st.subject} are not {st.predicate}", end="")

    def build_statement(self, logic):
        logic = logic.replace(" ", "").replace(",", "")
        neg = False
        only_sub = False

        for i in range(len(logic)):
            ch = logic[i]

            if ch == '!':
                neg = True
            elif ch == '&':
                print(" and ", end="")
            elif ch == '|':
                print(" or ", end="")
            elif ch == '*':
                print(" if and only if ", end="")
            elif ch == '~':
                print(" implies ", end="")
            elif ch == 'A':
                print("For all ", end="")
                only_sub = True
            elif ch == 'E':
                print("For some ", end="")
                only_sub = True
            elif ch in ['(', ')']:
                continue
            else:
                if only_sub:
                    print(f"{self.map[ch].subject}, ", end="")
                    only_sub = False
                elif neg:
                    self.print_neg_statement(ch)
                    neg = False
                else:
                    self.print_statement(ch)
        print(".")  


relation = Relation()

relation.add("s", "students", "brilliant", singular=False)
relation.add("a", "Real CR", "student")
relation.add("b", "Real CR", "lazy")

relation.build_statement("A(s) s")      
relation.build_statement("a & !b ~ a") 


For all students, students are brilliant.
Real CR is student and Real CR is not lazy implies Real CR is student.


In [ ]:
from pyDatalog import pyDatalog

pyDatalog.clear()


pyDatalog.create_terms('Brilliant, Lazy, Student, RealCR, X, Y')

+Student('students')
+Brilliant('students')
+RealCR('Real CR')
+Student('Real CR')
+Lazy('Real CR')

print("Is students brilliant?", bool(Brilliant('students')))
print("Is Real CR a student?", bool(Student('Real CR')))
print("Is Real CR lazy?", bool(Lazy('Real CR')))


Brilliant(Y) <= (Student(Y) & (Y=='Real CR'))

print("Then Real CR brilliant? =>", bool(Brilliant('Real CR')))


Is students brilliant? True
Is Real CR a student? True
Is Real CR lazy? True
Then Real CR brilliant? => True


In [ ]:

def draw_table(li, *args):
    length = len(" | ".join(args))
    print("-" * length)
    print(" | ".join(args))
    print("-" * length)
    for row in li:
        print(" | ".join(str(i) for i in row))
    print("-" * length)


draw_table([["1", "1", "1"], 
            ["1", "0", "0"], 
            ["0", "1", "0"], 
            ["0", "0", "0"]], "A", "B", "A AND B")


def evaluate_expression(a, b, ex):
    """
    & = AND
    | = OR
    ! = NOT
    ~ = IMPLIES
    * = IF AND ONLY IF
    """
    if (ex == '!'):
        return 0 if a == 1 else 1
    elif (ex == '&'):
        return 1 if a == 1 and b == 1 else 0
    elif (ex == '|'):
        return 1 if a == 1 or b == 1 else 0
    elif (ex == '~'):
        return 0 if a == 1 and b == 0 else 1
    elif (ex == '*'):
        return 1 if (a == 1 and b == 1) or (a == 0 and b == 0) else 0


N = 2 
arr = [list(map(int, bin(i)[2:].zfill(N))) for i in range(2**N)]

for i in arr:
    i.append(int(str(evaluate_expression(i[0], i[1], '*'))))

draw_table(arr, "A", "B", "ONLY IF")


---------------
A | B | A AND B
---------------
1 | 1 | 1
1 | 0 | 0
0 | 1 | 0
0 | 0 | 0
---------------
---------------
A | B | ONLY IF
---------------
0 | 0 | 1
0 | 1 | 0
1 | 0 | 0
1 | 1 | 1
---------------


In [ ]:
from collections import deque


class Relation(Relation):
    def __init__(self):
        super().__init__()
        self.operand_val = {}

    def map_operands(self, op, val):
        self.operand_val[op] = val

    def evalutate_expression(self, exp):
        """
        & = AND
        | = OR
        ! = NOT
        ~ = IMPLIES
        * = IF AND ONLY IF
        """
        exp = exp.replace(" ", "") 
        exp = exp.replace(",", "")  

        operators = deque()
        operands = deque()
        negative = False

        for i in exp:
            if i in ['&', '|', '~', '*']:
                operators.append(i)
            elif i == '!':
                negative = True
            else:
                if negative:
                    operands.append(evaluate_expression(self.operand_val[i], 0, '!'))
                    negative = False
                else:
                    operands.append(i)

        operands = deque(self.operand_val[i] if (i != 0 and i != 1) else i for i in operands)

        while operators:
            op = operators.popleft()
            a = operands.popleft()
            b = operands.popleft()
            operands.appendleft(evaluate_expression(a, b, op))

        return operands.popleft()


relation = Relation()

relation.add("a", "sensei", "student")
relation.add("b", "sensei", "brilliant")

relation.map_operands("a", 1)
relation.map_operands("b", 0)

print(relation.evalutate_expression("a | b & !a ~ b"))


1


In [ ]:

def AND(x, y): return x & y
def OR(x, y): return x | y
def NOT(x): return 1 - x
def IMPLIES(x, y): return (not x) or y
def IFF(x, y): return x == y


print("-----------------------------")
print("a | b | a OR NOT(b) IMPLIES b")
print("-----------------------------")
for A in [0,1]:
    for B in [0,1]:
        result = IMPLIES(OR(A, NOT(B)), B)
        print(f"{A} | {B} | {int(result)}")
print("-----------------------------")


-----------------------------
a | b | a OR NOT(b) IMPLIES b
-----------------------------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 0
1 | 1 | 1
-----------------------------


# 2019-20 

In [ ]:

def evaluate(a, b, op):
    if op == '!': return 1 - a
    if op == '&': return a & b
    if op == '|': return a | b
    if op == '~': return 1 if not(a==1 and b==0) else 0
    if op == '*': return 1 if a==b else 0

def draw_table(rows, *headers):
    print(" | ".join(headers))
    print("-" * (len(" | ".join(headers))))
    for row in rows:
        print(" | ".join(str(x) for x in row))
    print("-" * (len(" | ".join(headers))))
    print("\n")

def truth_table(variables, expression):
    N = len(variables)
    arr = [list(map(int, bin(i)[2:].zfill(N))) for i in range(2**N)]
    table = []
    for row in arr:

        val_map = {var: row[i] for i, var in enumerate(variables)}
        expr = expression

        while '!' in expr:
            idx = expr.find('!')
            var = expr[idx+1]
            val = 1 - val_map[var]
            expr = expr[:idx] + str(val) + expr[idx+2:]

        for var in variables:
            expr = expr.replace(var, str(val_map[var]))
      
        while any(op in expr for op in ['&','|','~','*']):
            for op in ['&','|','~','*']:
                if op in expr:
                    for i in range(len(expr)-2):
                        if expr[i] in '01' and expr[i+1] == op and expr[i+2] in '01':
                            a = int(expr[i])
                            b = int(expr[i+2])
                            val = evaluate(a,b,op)
                            expr = expr[:i] + str(val) + expr[i+3:]
                            break
        table.append(row + [int(expr)])
    draw_table(table, *(variables + ["Result"]))


print("a. It is raining outside if and only if it is a cloudy day (R * C)")
truth_table(['R','C'], 'R*C')

print("b. If you get 100 on final exam then you earn an A (F ~ A)")
truth_table(['F','A'], 'F~A')

print("c. Take either 2 Advil or 3 Tylenol (T2 | T3)")
truth_table(['T2','T3'], 'T2|T3')

print("d. She studied hard or she is extremely bright (S | B)")
truth_table(['S','B'], 'S|B')

print("e. I am a rock and I am an island (R & I)")
truth_table(['R','I'], 'R&I')


a. It is raining outside if and only if it is a cloudy day (R * C)
R | C | Result
--------------
0 | 0 | 1
0 | 1 | 0
1 | 0 | 0
1 | 1 | 1
--------------


b. If you get 100 on final exam then you earn an A (F ~ A)
F | A | Result
--------------
0 | 0 | 1
0 | 1 | 1
1 | 0 | 0
1 | 1 | 1
--------------


c. Take either 2 Advil or 3 Tylenol (T2 | T3)
T2 | T3 | Result
----------------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 1
----------------


d. She studied hard or she is extremely bright (S | B)
S | B | Result
--------------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 1
--------------


e. I am a rock and I am an island (R & I)
R | I | Result
--------------
0 | 0 | 0
0 | 1 | 0
1 | 0 | 0
1 | 1 | 1
--------------




# 2022 sessional 

In [ ]:

Parent = {
    ("Jhon", "Mary"),
    ("Mary", "Joe")
}

def is_grandparent(x, z):
    for (a, b) in Parent:
        if a == x:  
            for (c, d) in Parent:
                if b == c and d == z: 
                    return True
    return False

print("Parent(Jhon, Mary):", ("Jhon", "Mary") in Parent)
print("Parent(Mary, Joe):", ("Mary", "Joe") in Parent)
print("Grandparent(Jhon, Joe):", is_grandparent("Jhon", "Joe"))


Parent(Jhon, Mary): True
Parent(Mary, Joe): True
Grandparent(Jhon, Joe): True


In [ ]:

from pyDatalog import pyDatalog 


pyDatalog.clear()


values = [True, False]

def truth_table(expr, vars, name):
    print(f"\nTruth Table for: {name}")
    header = " | ".join(vars) + " | Result"
    print(header)
    print("-" * len(header))
    for combo in [(x,y) if len(vars)==2 else (x,y,z) if len(vars)==3 else (x,) for x in values for y in (values if len(vars)>=2 else [None]) for z in (values if len(vars)==3 else [None])]:

        var_dict = {}
        for i,v in enumerate(vars):
            var_dict[v] = combo[i]
        res = expr(**var_dict)
        row = " | ".join(str(var_dict[v]) for v in vars) + " | " + str(res)
        print(row)

def expr_a(R, C):
    return (R == C)  

truth_table(expr_a, ["R", "C"], "It is raining iff cloudy")


def expr_b(F, A):
    return (not F) or A   

truth_table(expr_b, ["F", "A"], "Final=100 -> Earn A")

def expr_c(Ad, Ty):
    return Ad or Ty   # OR

truth_table(expr_c, ["Ad", "Ty"], "2 Advil OR 3 Tylenol")

def expr_d(S, B):
    return S or B

truth_table(expr_d, ["S", "B"], "Studied OR Bright")

def expr_e(Rock, Island):
    return Rock and Island

truth_table(expr_e, ["Rock", "Island"], "Rock AND Island")



Truth Table for: It is raining iff cloudy
R | C | Result
--------------
True | True | True
True | False | False
False | True | False
False | False | True

Truth Table for: Final=100 -> Earn A
F | A | Result
--------------
True | True | True
True | False | False
False | True | True
False | False | True

Truth Table for: 2 Advil OR 3 Tylenol
Ad | Ty | Result
----------------
True | True | True
True | False | True
False | True | True
False | False | False

Truth Table for: Studied OR Bright
S | B | Result
--------------
True | True | True
True | False | True
False | True | True
False | False | False

Truth Table for: Rock AND Island
Rock | Island | Result
----------------------
True | True | True
True | False | False
False | True | False
False | False | False
